![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Similitud y normalización de textos

En este notebook aprenderá a calcular la similitud entre diferentes textos y a normalizarlos usando sklearn y [nltk](https://www.nltk.org/).

Este notebook tiene una licencia de [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). Un agradecimiento especial para [
Adrien sieg](https://medium.com/@adriensieg/text-similarities-da019229c894)

## Instrucciones Generales

La similitud y normalización de textos son tecnicas del procesamiento de lenguaje natural. Mientras que la similitud permite identificar que tan similares son un par de textos, la normalización permite convertir una palabra en su forma más básica.

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá a obtener la similitud entre dos textos usando diferentes métricas. En la segunda parte, normalizará el texto del set de noticias populares de UCI, eliminando stopwords y haciedo stemming y lematización. Para conocer más detalles de la base, puede ingresar al siguiente [vínculo](https://archive.ics.uci.edu/ml/datasets/online+news+popularity#).
   
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 

In [16]:
import warnings
warnings.filterwarnings('ignore')

## Similitud de texto

### Similitud de Jaccard
La similitud de Jaccard se define como el tamaño de la intersección dividido por el tamaño de la unión de dos conjuntos.

In [17]:
# Definición función de similitud de Jaccard que recibe como parámetros dos textos y retorna su similitud
def jaccard_similarity(query, document):
    # Calculo de la intersección
    intersection = set(query.split()).intersection(set(document.split()))
    # Calculo de la unión
    union = set(query.split()).union(set(document.split()))
    return len(intersection)/len(union)

In [18]:
# Definición de oraciones para calculo de similitud
s1 = "La intelingencia artificial ayuda a resolver los problemas mas complejos"
s2 = "La inteligencia artificial está creciendo rápidamente y esto puede acarrear diferentes problemas"

In [19]:
# Impresión de la similitud de Jaccard entre las dos frases
jaccard_similarity(s1, s2)

0.15789473684210525

### Similitud de coseno

La similitud del coseno calcula la similitud midiendo el coseno del ángulo entre dos vectores.

In [20]:
# Importación librerías
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
import numpy as np

#### Similitud de coseno CountVectorizer
Al vectorizar con CountVectorizer, este tiene la limitación que palabras de un carácter no se consideran dentro del vocabulario, por ejemplo las palabras 'a' e 'y'. Con esto se tiene:

In [21]:
# Definición función de similitud de Coseno que recibe como parámetros dos textos y retorna su similitud
def cosine_distance_countVectorizer(s1, s2):

    # Uso de CountVectorizer para obtener vectores de una frase
    vect = CountVectorizer(binary=True)
    X_dtm = vect.fit_transform([s1, s2]).todense()
    
    return 1-cosine(X_dtm[0], X_dtm[1])

In [22]:
# Impresión de la similitud de coseno entre las dos frases definidas anteriormente
cosine_distance_countVectorizer(s1, s2)

ValueError: Input vector should be 1-D.

#### Similitud de coseno manual
Al realizar la creación los vectores de la frase manualmente se garantiza que se consideran todas las palabras. Con esto:

In [ ]:
def obtener_vectores(union, s1, s2):

    s1_l = []
    s2_l = []
    
    for palabra in union:
        if palabra in s1.split():
            s1_l.append(1)
        else:
            s1_l.append(0)

        if palabra in s2.split():
            s2_l.append(1)
        else:
            s2_l.append(0)

    return s1_l, s2_l
        
# Definición función de similitud de Coseno que recibe como parámetros dos textos y retorna su similitud
def cosine_distance_manual(s1, s2):
    
    union = list(set(s1.split()).union(set(s2.split())))
    
    s1_v, s2_v = obtener_vectores(union, s1, s2)
    
    return 1-cosine(s1_v, s2_v)

In [ ]:
# Impresión de la similitud de coseno entre las dos frases definidas anteriormente
cosine_distance_manual(s1, s2)

0.27386127875258304

La diferencia entre las distancias de coseno se obtiene por la forma de vectorizar, consideren esta segunda para el desarrollo del quiz. Los invitamos a que entiendan con detalle que hace la función manual.

### Codificación de Oraciones y Similitud de Coseno

La codificación de oraciones es una de las representaciones más populares del vocabulario de documentos. Es capaz de capturar el contexto de una palabra en un documento, la similitud semántica y sintáctica, la relación con otras palabras, etc. 

Para esta sección del notebook instale la libreria tensorflow y tensorflow_hub (si aun no las ha instalado) con el comando *!pip install tensorflow* y *!pip install tensorflow_hub* respectivamente.

In [ ]:
#pip install tensorflow_hub #tensorflow

In [ ]:
# Importación librerías
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub

In [ ]:
# Importación el módulo TF Hub del Universal Sentence Encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
embed = hub.Module(module_url)


In [ ]:
# Codificación de las frases anteriormente definidas con la libreria tensorflow
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings = session.run(embed([s1, s2]))

DataLossError: Graph execution error:

Detected at node 'checkpoint_initializer_3' defined at (most recent call last):
    File "c:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    Fil

In [ ]:
#Impresión de las codificaciones
sentences_embeddings

In [ ]:
# Impresión de la similitud de coseno entre las dos frases definidas anteriormente usando codificación de oraciones
1-cosine(sentences_embeddings[0], sentences_embeddings[1])

NameError: name 'sentences_embeddings' is not defined

## Normalización de textos

In [ ]:
# Importación librerías
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [ ]:
# Carga de datos de archivos .csv
df = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/mashable_texts.csv', index_col=0)
df.head()

In [ ]:
# Separación de variable de interés (y)
y = df.shares
y.describe()

In [ ]:
# Categoización de la variable de interés (y)
y = pd.cut(y, [0, 893, 1200, 2275, 63200], labels=[0, 1, 2, 3])
y.value_counts()

In [ ]:
# Definición de variable de interés en el dataframe
df['y'] = y

In [ ]:
# Definición de variables predictoras
X = df.text

In [ ]:
# Definición de función que recibe un texto vectorizado y calcula el acurracy de un modelo Naive Bayes 
def tokenize_test(vect):
    X_dtm = vect.fit_transform(X)
    print('Features: ', X_dtm.shape[1])
    nb = MultinomialNB()
    print(pd.Series(cross_val_score(nb, X_dtm, y, cv=10)).describe())

### Eliminación de stopwords

In [ ]:
# Eliminación de stopwords al usar el parámetro 'stop_words' de la función CountVectorizer()
vect_no_stopw = CountVectorizer(stop_words='english')

In [ ]:
# Impresión de stopwords del texto
print(vect_no_stopw.get_stop_words())

In [ ]:
# Desempeño del modelo sin considerar stopwords
tokenize_test(vect_no_stopw)

### Stemming

Stemming es un preprocesamiento del texto en el que para cada palabra se obtiene su raíz o en inglés stem.

In [ ]:
# Inicialización de stemmer
stemmer = SnowballStemmer('english')

In [ ]:
# Creación de matrices de documentos usando CountVectorizer a partir de X
vect = CountVectorizer()
vect.fit(X)

In [ ]:
# Definiicón de lista con vocabulario de la matriz de documentos
words = list(vect.vocabulary_.keys())[:100]

In [ ]:
# Obtención e impresión de los stem de cada palabra de la lista
print([stemmer.stem(word) for word in words])

### Lematización

La lemmatización es un proceso en el que se busca el lema de cada palabra de un texto, siendo un lema la forma base o de diccionario de una palabra.

In [ ]:
# Importación de librerias
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

In [ ]:
# Obtención e impresión de los lemas de cada palabra de la lista asumiendo que cada palabra es un sustantivo
print([wordnet_lemmatizer.lemmatize(word) for word in words])

In [ ]:
# Obtención e impresión de los lemas de cada palabra de la lista asumiendo que cada palabra es un verbo
print([wordnet_lemmatizer.lemmatize(word,pos='v') for word in words])

In [ ]:
# Definición de la función que tenga como parámetro texto y devuelva una lista de lemas
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

In [ ]:
# Creación de matrices de documentos usando CountVectorizer, usando el parámetro 'split_into_lemmas'
vect_lemas = CountVectorizer(analyzer=split_into_lemmas)

In [ ]:
# Desempeño del modelo al lematizar el texto
tokenize_test(vect_lemas)